In [10]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from IPython.display import display, Image
import os


# Load class indices
class_indices = {
    'Apple_Apple Scab': 0, 'Apple_Black Rot': 1, 'Apple_Cedar Apple Rust': 2, 'Apple_Healthy': 3,
    'Bell Pepper_Bacterial Spot': 4, 'Bell Pepper_Healthy': 5, 'Cherry_Healthy': 6, 'Cherry_Powdery Mildew': 7,
    'Corn (Maize)_Cercospora Leaf Spot': 8, 'Corn (Maize)_Common Rust': 9, 'Corn (Maize)_Healthy': 10,
    'Corn (Maize)_Northern Leaf Blight': 11, 'Grape_Black Rot': 12, 'Grape_Esca (Black Measles)': 13,
    'Grape_Healthy': 14, 'Grape_Leaf Blight': 15, 'Peach_Bacterial Spot': 16, 'Peach_Healthy': 17,
    'Potato_Early Blight': 18, 'Potato_Healthy': 19, 'Potato_Late Blight': 20, 'Strawberry_Healthy': 21,
    'Strawberry_Leaf Scorch': 22, 'Tomato_Bacterial Spot': 23, 'Tomato_Early Blight': 24, 'Tomato_Healthy': 25,
    'Tomato_Late Blight': 26, 'Tomato_Septoria Leaf Spot': 27, 'Tomato_Yellow Leaf Curl Virus': 28
}

# Load trained TensorFlow model
model_path = 'student_model_final.h5'
loaded_model = tf.keras.models.load_model(model_path)

# Load plant disease dataset (recommendation CSV)
df = pd.read_csv("plant.csv")  

# Function to preprocess image
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to [0,1]
    return img_array

# Function to predict disease from an image
def predict_disease(model, img_path, class_indices):
    img_array = load_and_preprocess_image(img_path)
    
    # Get predictions
    preds = model.predict(img_array)
    
    # If the model returns multiple outputs, assume predictions are the second element
    if isinstance(preds, list) and len(preds) >= 2:
        preds = preds[1]
    
    # Convert to a NumPy array and flatten it to 1D
    preds = np.asarray(preds).flatten()
    
    # Get the predicted class index
    pred_idx = np.argmax(preds)
    
    # Invert class_indices to get class names
    inv_class_indices = {v: k for k, v in class_indices.items()}
    predicted_label = inv_class_indices.get(pred_idx, "Unknown Disease")
    
    print(f"🦠 Predicted Disease: {predicted_label}")
    return predicted_label

# Function to get recommendations for a predicted disease
def get_recommendation(disease_name):
    # Use strip() and lower() to remove extra spaces and normalize case
    disease_info = df[df["Disease"].str.strip().str.lower() == disease_name.strip().lower()]
    
    if disease_info.empty:
        print("No recommendation found for this disease. The leaf is healthy.")
        return
    
    # Extract relevant details
    plant = disease_info.iloc[0]["Plant"]
    symptoms = disease_info.iloc[0]["Symptoms"]
    causes = disease_info.iloc[0]["Causes"]
    treatment = disease_info.iloc[0]["Recommended Treatments"]
    prevention = disease_info.iloc[0]["Preventive Measures"]
    organic_treatment = disease_info.iloc[0]["Organic Treatment Options"]
    alternative_prevention = disease_info.iloc[0]["Alternative Prevention Methods"]
    procedure_video = disease_info.iloc[0]["Procedure Video Link"]
    disease_image = disease_info.iloc[0]["Graphical Representation"]

    # Display recommendation details
    recommendation = f"""
    🌱 **Plant:** {plant}
    🦠 **Disease:** {disease_name}
    🔬 **Symptoms:** {symptoms}
    ⚠ **Causes:** {causes}
    💊 **Recommended Treatment:** {treatment}
    ✅ **Preventive Measures:** {prevention}
    🌿 **Organic Treatment Options:** {organic_treatment}
    🔄 **Alternative Prevention Methods:** {alternative_prevention}
    🎥 **Procedure Video:** {procedure_video}
    """
    
    print(recommendation)
    
    # Display image if available
    if pd.notna(disease_image) and disease_image.startswith("http"):
        display(Image(url=disease_image, width=400))

new_image_path = 'apple.png'

predicted_disease = predict_disease(loaded_model, new_image_path, class_indices)

get_recommendation(predicted_disease)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 652ms/step
🦠 Predicted Disease: Bell Pepper_Healthy
No recommendation found for this disease. The leaf is healthy.


In [34]:
new_image_path = '/kaggle/input/test-leaves/AppleHealthy.jpg'

predicted_disease = predict_disease(loaded_model, new_image_path, class_indices)

get_recommendation(predicted_disease)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
🦠 Predicted Disease: Apple_Healthy
No recommendation found for this disease. The leaf is healthy.


In [35]:
new_image_path = '/kaggle/input/test-leaves/BellPepperBS.jpg'

predicted_disease = predict_disease(loaded_model, new_image_path, class_indices)

get_recommendation(predicted_disease)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
🦠 Predicted Disease: Bell Pepper_Bacterial Spot

    🌱 **Plant:** Bell Pepper
    🦠 **Disease:** Bell Pepper_Bacterial Spot
    🔬 **Symptoms:** Water-soaked lesions on leaves & fruit
    ⚠ **Causes:** Xanthomonas campestris (bacteria)
    💊 **Recommended Treatment:** Copper-based bactericide spray
    ✅ **Preventive Measures:** Use disease-free seeds, avoid overhead watering
    🌿 **Organic Treatment Options:** Copper hydroxide spray
    🔄 **Alternative Prevention Methods:** Crop rotation with non-host plants
    🎥 **Procedure Video:** https://youtu.be/t1eIkOhfYbk?si=ejhJKKbogygTPIxn  /  https://youtu.be/XPtRY8pANzU?si=J1cs8NJsQaj_JHLB
    


In [36]:
new_image_path = '/kaggle/input/test-leaves/TomatoEarlyBlight.jpg'

predicted_disease = predict_disease(loaded_model, new_image_path, class_indices)

get_recommendation(predicted_disease)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
🦠 Predicted Disease: Tomato_Early Blight

    🌱 **Plant:** Tomato
    🦠 **Disease:** Tomato_Early Blight
    🔬 **Symptoms:** Dark, circular spots on lower leaves
    ⚠ **Causes:** Alternaria solani (fungus)
    💊 **Recommended Treatment:** Use chlorothalonil or mancozeb
    ✅ **Preventive Measures:** Remove infected leaves, stake plants
    🌿 **Organic Treatment Options:** Neem oil, Sulfur spray
    🔄 **Alternative Prevention Methods:** Increase air circulation
    🎥 **Procedure Video:** https://youtu.be/NtVqk3iPT44?si=zYwDuo6quyMQvYd4 / https://youtu.be/NtVqk3iPT44?si=zYwDuo6quyMQvYd4
    


In [37]:
new_image_path = '/kaggle/input/test-leaves/Potato Healthy.jpg'

predicted_disease = predict_disease(loaded_model, new_image_path, class_indices)

get_recommendation(predicted_disease)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
🦠 Predicted Disease: Potato_Healthy
No recommendation found for this disease. The leaf is healthy.


In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from IPython.display import display, Image
import os

# Load class indices
class_indices = {
    'Apple_Apple Scab': 0, 'Apple_Black Rot': 1, 'Apple_Cedar Apple Rust': 2, 'Apple_Healthy': 3,
    'Bell Pepper_Bacterial Spot': 4, 'Bell Pepper_Healthy': 5, 'Cherry_Healthy': 6, 'Cherry_Powdery Mildew': 7,
    'Corn (Maize)_Cercospora Leaf Spot': 8, 'Corn (Maize)_Common Rust': 9, 'Corn (Maize)_Healthy': 10,
    'Corn (Maize)_Northern Leaf Blight': 11, 'Grape_Black Rot': 12, 'Grape_Esca (Black Measles)': 13,
    'Grape_Healthy': 14, 'Grape_Leaf Blight': 15, 'Peach_Bacterial Spot': 16, 'Peach_Healthy': 17,
    'Potato_Early Blight': 18, 'Potato_Healthy': 19, 'Potato_Late Blight': 20, 'Strawberry_Healthy': 21,
    'Strawberry_Leaf Scorch': 22, 'Tomato_Bacterial Spot': 23, 'Tomato_Early Blight': 24, 'Tomato_Healthy': 25,
    'Tomato_Late Blight': 26, 'Tomato_Septoria Leaf Spot': 27, 'Tomato_Yellow Leaf Curl Virus': 28
}

# Load trained TensorFlow model
model_path = 'leaf_vs_non-leaf.h5'
model = tf.keras.models.load_model(model_path)

def load_and_preprocess_image(img_path, target_size=(224, 224)):  # Correct size!
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to [0,1]
    return img_array

img_path = 'bell_pepper.webp'  # Make sure to use the correct path to your test image
img_array = load_and_preprocess_image(img_path)

preds = model.predict(img_array)
    
print(f"🦠 Prediction: {preds}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
🦠 Prediction: [[9.9999976e-01 1.8987058e-07]]
